In [12]:
import geemap
import folium
import ee
import os


import numpy as np
import palettable 
import numpy as np
from Satellite import *
from Simulator import *
service_account = "jims-948@jimssatellites.iam.gserviceaccount.com"
key_path = 'jimssatellites-25b0240d5cf2.json'
credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials=credentials)



count = 0
import csv
sat1 = Satellite("Satellites/sat1.txt", "SAT1")
sat2 = Satellite("Satellites/sat2.txt", "SAT2")
sat3 = Satellite("Satellites/sat3.txt", "SAT3")
sat4 = Satellite("Satellites/sat4.txt", "SAT4")
sat5 = Satellite("Satellites/sat5.txt", "SAT5")
sat6 = Satellite("Satellites/sat6.txt", "SAT6")  
sat7 = Satellite("Satellites/sat7.txt", "SAT7")
sat8 = Satellite("Satellites/sat8.txt", "SAT8")
sat9 = Satellite("Satellites/sat9.txt", "SAT9")  
sat10 = Satellite("Satellites/sat10.txt", "SAT10")
sat11 = Satellite("Satellites/sat11.txt", "SAT11")
sat12 = Satellite("Satellites/sat12.txt", "SAT12")

satellites = [sat1,sat2,sat3,sat4,sat5,sat6,sat7,sat8,sat9,sat10,sat11,sat12]

count = 0
mappingArray = []
for sat in satellites:
    csv_file = "RESULTS_"+sat.name+".csv"
    count 
    with open(csv_file, mode='r', newline='') as file:
        reader = csv.reader(file)  
        i = 0
        for row in reader:
            coord1a, coord1b,  coord4a, coord4b, error = [float(param) for param in row]
            roi = ee.Geometry.Rectangle([[coord1a, coord1b],  [coord4a, coord4b]])
            newFeature = ee.Feature(roi, {'name': str(count)})
            count = count+1
            mappingArray.append(newFeature)
            count+=1




In [16]:

# Create an interactive map
Map = geemap.Map(center=[0, 0], zoom=2)
mappedArea = ee.FeatureCollection(mappingArray)
NSW = ee.Geometry.Rectangle([[144.81577841465915, -36.42295520530404], [155.81727106335805, -27.806830550620564]])
# Create an image collection for Sentinel-2 data
start_date = '2019-06-01'
end_date = '2020-05-31'
sentinel2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                        .filterDate(ee.Date(start_date), ee.Date(end_date))
                        .filterBounds(mappedArea)
                        .select('B.*|MSK_CLDPRB|SCL')
)
burnedArea = ee.Image('projects/jimssatellites/assets/fires')
nir = sentinel2_collection.select('B8')  # Near-Infrared Band (B8)
swir1 = sentinel2_collection.select('B11')  # Short-Wave Infrared Band 1 (B11)
swir2 = sentinel2_collection.select('B12')  # Short-Wave Infrared Band 2 (B12)


##CREATE THE BLACK MASK

roi = ee.Geometry.Rectangle([ 146,-35.36,  158.6, -30.83])
#I have no idea why, but for some reason this just paints everything so I'm gonna cop for now
mask = ee.Image(0).toInt8().paint(roi, 1)
fc = ee.FeatureCollection(mappingArray)
# Fill and outline the polygons in two colors
filled = mask.paint(fc, 2)
both = filled.paint(fc, 1, 5)
# Mask off everything that matches the fill color.
result = both.mask(filled.neq(2))


nir_palette = ['000000', '330000', '660000', '990000', 'CC0000', 'FF0000', 'FF1900', 'FF3200', 'FF4B00', 'FF6400',
              'FF7D00', 'FF9600', 'FFAF00', 'FFC800', 'FFE100', 'FFFF00']
# Create a visualization palette
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B11', 'B12']
}
# Define visualization parameters for each band
nir_vis_params = {
    'min': 0,
    'max': 10000,
    'palette': nir_palette  # Apply the custom color palette for NIR
}
#Map.addLayer(sentinel2_collection, vis_params, 'Sentinel-2 Image')
#Map.addLayer(nir, nir_vis_params, 'NIR')

#Short wave infrared
#Map.addLayer(swir1, nir_vis_params, 'SWIR1')


burnedVisParams = {
    'min': 0,
    'max': 1000,
    'palette': [  '000004', '080005', '0e0b05', '161007', '1b1206', '201606', '251d07',
  '292606', '2d2c08', '313307', '353a08', '393f0a', '3d440b', '41490c',
  '444e0d', '47520e', '4a560f', '4c5b10', '4f5f11', '526311', '556812',
  '576c13', '596f14', '5b7315', '5d7616', '5f7a17', '617d18', '638119',
  '65851a', '67881b', '698b1c', '6a8e1d', '6c921e', '6d9520', '6f9821',
  '719b22', '729e23', '74a124', '75a425', '77a726', '78aa28', '79ad29',
  '7ba02a', '7ca32c', '7da62d', '7ea92f', '7fac30', '80af32', '81b234',
  '82b536', '83b838', '84bb3a', '85be3b', '86c13d', '87c440', '88c742',
  '89ca44', '8acc47', '8bcd49', '8cd04b', '8dd34e', '8ed550', '8fd753',
  '90da55', '91dc57', '92df5a', '93e25c', '94e45f', '95e762', '96e964',
  '97ec67', '98ee6a', '99f06c', '9af36f', '9bf572', '9cf775', '9df978',
  '9efc7b', 'a0fe7e']
}


#Burned Area
Map.addLayer(burnedArea, burnedVisParams)

#Black mask
Map.addLayer(result, {
    'palette': '000000,FF0000',
    'max': 1,
    'opacity': 1.0
}, 'Opaque White Rectangle')
#Map.addLayer(swir2, {'min': 0, 'max': 3000}, 'SWIR2')
Map


The provided color (000000,FF0000) is invalid. Using the default black color.
'#000000,FF0000' is not in web format. Need 3 or 6 hex digit.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…